In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import seaborn as sns
import re

In [2]:
df = pd.read_excel(r'D:\IT\mywork\Project\KBO-Analysis\dataset\kbo_batter.xlsx', encoding='utf-8', index=False, dtype={'Season': object})
df

,Name,Season,Birth,Team,Position,WAR,G,PA,AB,R,...,RBI,SB,CB,BB,HBP,IBB,SO,DP,SH,SF
0,백인천,82,1943-11-27,M,DH,5.88,72,298,250,55,...,64,11,3,42,3,10,17,5,0,3
1,윤동균,82,1949-07-02,O,LF,4.13,77,334,284,54,...,47,9,2,45,1,2,27,3,0,4
2,이만수,82,1958-09-19,삼,C,4.12,78,322,270,46,...,51,6,2,46,4,0,27,13,0,1
3,김봉연,82,1952-01-13,해,DH,3.68,74,304,269,55,...,52,7,2,28,6,3,16,8,0,1
4,김우열,82,1949-09-09,O,CF,3.46,62,255,210,43,...,41,14,4,38,6,6,25,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9280,이성열,20,1984-07-13,한,DH,-0.93,79,248,227,23,...,34,1,1,16,5,0,69,4,0,0
9281,김규성,20,1997-03-08,K,2B,-1.01,103,182,157,21,...,11,2,1,13,1,0,40,1,11,0
9282,김태진,20,1995-10-07,NK,3B,-1.03,82,280,263,28,...,23,7,4,13,1,0,45,2,2,1
9283,박찬호,20,1995-06-05,K,SS,-1.15,141,530,479,63,...,36,15,8,36,0,0,87,14,12,3


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9285 entries, 0 to 9284
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      8204 non-null   object 
 1   Season    8204 non-null   object 
 2   Birth     9285 non-null   object 
 3   Team      9285 non-null   object 
 4   Position  7775 non-null   object 
 5   WAR       9285 non-null   float64
 6   G         9285 non-null   int64  
 7   PA        9285 non-null   int64  
 8   AB        9285 non-null   int64  
 9   R         9285 non-null   int64  
 10  H         9285 non-null   int64  
 11  2B        9285 non-null   int64  
 12  3B        9285 non-null   int64  
 13  HR        9285 non-null   int64  
 14  TB        9285 non-null   int64  
 15  RBI       9285 non-null   int64  
 16  SB        9285 non-null   int64  
 17  CB        9285 non-null   int64  
 18  BB        9285 non-null   int64  
 19  HBP       9285 non-null   int64  
 20  IBB       9285 non-null   int6

In [4]:
tmp = df[df.isna().any(axis=1)] # NA를 포함한 행 중에서
tmp = tmp[tmp.WAR > 1]  # 어느정도 활약한 선수 filtering
tmp

,Name,Season,Birth,Team,Position,WAR,G,PA,AB,R,...,RBI,SB,CB,BB,HBP,IBB,SO,DP,SH,SF
554,최홍석,86,1960-03-28,M,NaN,1.03,79,152,142,13,...,19,1,0,5,2,0,21,1,2,1
875,이광근,88,1962-01-24,태,NaN,1.79,67,234,205,20,...,25,0,2,18,2,0,26,3,8,1
1073,최동창,89,1965-06-07,O,NaN,1.16,50,116,97,10,...,21,0,2,14,1,0,16,2,0,4
1243,이광근,90,1962-01-24,태,NaN,1.67,66,189,159,25,...,18,1,1,25,3,1,17,5,2,0
1440,정문언,91,1962-08-13,태,NaN,2.42,114,379,335,32,...,33,1,6,39,2,2,60,5,2,1
1452,정성룡,91,1964-05-19,삼,NaN,1.56,77,157,132,22,...,12,2,3,22,1,0,23,1,2,0
1467,최동창,91,1965-06-07,O,NaN,1.16,88,238,206,19,...,21,1,2,20,10,1,58,5,2,0
1674,정문언,92,1962-08-13,태,NaN,1.97,78,239,217,27,...,24,1,5,21,0,1,36,5,0,1
1690,김건우,92,1963-08-30,L,NaN,1.33,54,214,196,28,...,30,2,0,16,0,0,31,5,0,2
1695,윤용하,92,1968-06-07,삼,NaN,1.16,77,143,131,14,...,18,0,2,9,1,2,24,1,1,1


In [ ]:
정문언, 로마이어 = 지타
이경복, 하득인 = 2루수
엔젤 = 3루수
김건우, 김병조 = 1루수
나머지 = 외야수

In [5]:
RF = ['최동창', '이시온', '박재용', '황성관', '윤용하', '최홍석', '정성룡', '지화선', '펨버튼', '잉글린', '오리어리', '고동진']
CF = ['이광근']
LF = ['박재벌']
DH = ['정문언', '로마이어']
B2 = ['이경복', '하득인']
B3 = ['엔젤']
B1 = ['김건우', '김병조']

In [6]:
for name in tmp.Name:
    if name[:-2] in RF:
        df.loc[df.Name == name, 'Position'] = 'RF'
        
    elif name[:-2] in LF:
        df.loc[df.Name == name, 'Position'] = 'LF'
        
    elif name[:-2] in LF:
        df.loc[df.Name == name, 'Position'] = 'LF'
        
    elif name[:-2] in DH:
        df.loc[df.Name == name, 'Position'] = 'DH'
        
    elif name[:-2] in B2:
        df.loc[df.Name == name, 'Position'] = '2B'
        
    elif name[:-2] in B3:
        df.loc[df.Name == name, 'Position'] = '3B'
        
    elif name[:-2] in B1:
        df.loc[df.Name == name, 'Position'] = '1B'

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9285 entries, 0 to 9284
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      8204 non-null   object 
 1   Season    8204 non-null   object 
 2   Birth     9285 non-null   object 
 3   Team      9285 non-null   object 
 4   Position  7775 non-null   object 
 5   WAR       9285 non-null   float64
 6   G         9285 non-null   int64  
 7   PA        9285 non-null   int64  
 8   AB        9285 non-null   int64  
 9   R         9285 non-null   int64  
 10  H         9285 non-null   int64  
 11  2B        9285 non-null   int64  
 12  3B        9285 non-null   int64  
 13  HR        9285 non-null   int64  
 14  TB        9285 non-null   int64  
 15  RBI       9285 non-null   int64  
 16  SB        9285 non-null   int64  
 17  CB        9285 non-null   int64  
 18  BB        9285 non-null   int64  
 19  HBP       9285 non-null   int64  
 20  IBB       9285 non-null   int6

In [8]:
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Name,Season,Birth,Team,Position,WAR,G,PA,AB,R,...,RBI,SB,CB,BB,HBP,IBB,SO,DP,SH,SF
0,백인천,82,1943-11-27,M,DH,5.88,72,298,250,55,...,64,11,3,42,3,10,17,5,0,3
1,윤동균,82,1949-07-02,O,LF,4.13,77,334,284,54,...,47,9,2,45,1,2,27,3,0,4
2,이만수,82,1958-09-19,삼,C,4.12,78,322,270,46,...,51,6,2,46,4,0,27,13,0,1
3,김봉연,82,1952-01-13,해,DH,3.68,74,304,269,55,...,52,7,2,28,6,3,16,8,0,1
4,김우열,82,1949-09-09,O,CF,3.46,62,255,210,43,...,41,14,4,38,6,6,25,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6801,이성열,20,1984-07-13,한,DH,-0.93,79,248,227,23,...,34,1,1,16,5,0,69,4,0,0
6802,김규성,20,1997-03-08,K,2B,-1.01,103,182,157,21,...,11,2,1,13,1,0,40,1,11,0
6803,김태진,20,1995-10-07,NK,3B,-1.03,82,280,263,28,...,23,7,4,13,1,0,45,2,2,1
6804,박찬호,20,1995-06-05,K,SS,-1.15,141,530,479,63,...,36,15,8,36,0,0,87,14,12,3


In [9]:
df.to_excel(r'D:\IT\mywork\Project\KBO-Analysis\dataset\kbo_batter2.xlsx',  encoding='utf-8', index=False)

In [ ]:
# 개명한 선수들 이름 변경

In [10]:
rename = pd.read_excel(r'D:\IT\mywork\Project\KBO-Analysis\dataset\rename.xlsx', encoding='utf-8', index=False)
rename

,Year,Before,After,Birth,Team
0,2017,고영우,고장혁,1990-01-08,기
1,2017,오승택,오태곤,1991-11-18,롯
2,2015,김영관,김지성,1985-07-20,L
3,2015,에릭,해커,1983-03-26,N
4,2015,이승화,이우민,1982-05-04,롯
5,2015,이학준,이시찬,1985-02-25,한
6,2014,김희걸,김건한,1981-03-26,삼
7,2014,문현정,문수호,1983-06-04,N
8,2013,김민식,김주원,1991-08-30,S
9,2013,김상현,김태영,1980-04-07,기


In [12]:
for idx in df.index:
    row = df.loc[idx, ['Name', 'Birth']]
    name = row['Name']
    birth = row['Birth']
    if name in list(rename.Before) and birth in list(rename.Birth):
        change = ''.join(rename.loc[(rename['Before'] == name) & (rename['Birth'] == birth)].After)
        df.loc[idx, 'Name'] = change
        print(list(df.loc[idx]))

['류은재', '07', '1983-12-13', 'K', 'CF', 0.25, 10, 12, 12, 4, 4, 0, 0, 2, 10, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0]
['류은재', '08', '1983-12-13', 'K', 'CF', -0.16, 13, 29, 27, 2, 4, 0, 0, 0, 4, 0, 2, 0, 2, 0, 0, 8, 0, 0, 0]
['류은재', '11', '1983-12-13', 'K', 'LF', 0.32, 10, 14, 13, 3, 4, 1, 1, 2, 13, 7, 0, 0, 1, 0, 0, 5, 0, 0, 0]
['고장혁', '16', '1990-01-08', 'K', 'SS', -0.01, 63, 81, 70, 18, 15, 3, 0, 0, 18, 4, 3, 2, 7, 3, 0, 15, 1, 1, 0]
['오태곤', '16', '1991-11-18', '롯', 'DH', -0.09, 42, 135, 123, 17, 32, 6, 0, 3, 47, 7, 6, 2, 12, 0, 0, 37, 5, 0, 0]
['오태곤', '19', '1991-11-18', 'K', '1B', 0.1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
df[df['Name'].isin(['고장혁', '류은재', '오태곤'])]

,Name,Season,Birth,Team,Position,WAR,G,PA,AB,R,...,RBI,SB,CB,BB,HBP,IBB,SO,DP,SH,SF
3619,류은재,07,1983-12-13,K,CF,0.25,10,12,12,4,...,2,0,0,0,0,0,3,0,0,0
3868,류은재,08,1983-12-13,K,CF,-0.16,13,29,27,2,...,0,2,0,2,0,0,8,0,0,0
4385,류은재,11,1983-12-13,K,LF,0.32,10,14,13,3,...,7,0,0,1,0,0,5,0,0,0
4926,고장혁,13,1990-01-08,K,SS,-0.16,29,11,11,7,...,0,2,1,0,0,0,6,0,0,0
5067,오태곤,14,1991-11-18,롯,SS,0.14,57,50,45,10,...,5,2,1,5,0,0,18,0,0,0
5161,고장혁,14,1990-01-08,K,SS,-0.29,21,27,24,4,...,1,0,0,1,1,0,6,0,0,1
5239,오태곤,15,1991-11-18,롯,SS,1.50,122,353,327,57,...,43,15,4,17,4,0,100,9,4,1
5428,고장혁,15,1990-01-08,K,2B,-0.54,82,57,52,17,...,4,7,3,4,0,0,15,0,1,0
5612,고장혁,16,1990-01-08,K,SS,-0.01,63,81,70,18,...,4,3,2,7,3,0,15,1,1,0
5634,오태곤,16,1991-11-18,롯,DH,-0.09,42,135,123,17,...,7,6,2,12,0,0,37,5,0,0
